# Models API

使用 Models API 查看和访问 OpenAI 提供的预训练大语言模型

## List Models

列出当前可用的模型，并提供每个模型的基本信息，如所有者和可用性。

In [3]:
import os
from openai import OpenAI
client = OpenAI()

models = client.models.list()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ghp_JgOy****************************MBlk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
print(models)

#### 查看 OpenAI 最新提供的模型 API 信息

`models.data`: 目前OpenAI提供的大语言模型列表，列表中的每一项都对应着一个模型实例。

以`GPT-3.5-Turbo`模型为例，解释说明各项参数：

1. `created`: 这是模型创建的时间戳，单位为 Unix 时间戳（自1970年1月1日（00:00:00 GMT）以后的秒数）。
2. `id`: 这是模型的唯一标识符。在这个例子中，模型的 ID 是 "text-davinci-003"。
3. `object`: 这个字段表示的是当前对象的类型，在这个例子中，对象是 "model"，说明这个 JSON 对象是一个模型。
4. `owned_by`: 这个字段表示的是模型的所有者，在这个例子中，模型的所有者是 "openai-internal"。

In [ ]:
models.data

### 获取模型 ID 列表

In [ ]:
models.data[0].id

In [ ]:
model_list = [model.id for model in models.data]

In [ ]:
print(model_list)

## Retrieve Model

根据前面查询到当前支持的模型ID列表，获取指定模型实例，如`gpt-3.5-turbo`。

In [ ]:
from openai import OpenAI
client = OpenAI()

# 将模型 ID 传入 retrieve 接口
gpt_3 = client.models.retrieve("gpt-3.5-turbo")

In [ ]:
print(gpt_3)

### 获取指定模型，如 GPT-4V

In [ ]:
client.models.retrieve("gpt-4-vision-preview")

# 文本内容补全初探（Completions API）[Legacy]

使用 Completions API 实现各类文本生成任务


主要请求参数说明：


- **`model`** （string，必填）

  要使用的模型的 ID。可以参考 **模型端点兼容性表**。

- **`prompt`** （string or array，必填，Defaults to ）

  生成补全的提示，编码为字符串、字符串数组、token数组或token数组数组。

  注意，这是模型在训练过程中看到的文档分隔符，所以如果没有指定提示符，模型将像从新文档的开头一样生成。

- **`stream`** （boolean，选填，默认 false）

  当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容，即会不断地输出内容直到完成响应，流通过 `data: [DONE]` 消息终止。

- **`max_tokens`** （integer，选填，默认是 16）

  补全时要生成的最大 token 数。

  提示 `max_tokens` 的 token 计数不能超过模型的上下文长度。大多数模型的上下文长度为 2048 个token（最新模型除外，它支持 4096）

- **`temperature`** （number，选填，默认是1）

  使用哪个采样温度，在 **0和2之间**。

  较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。

  通常建议修改这个（`temperature` ）或 `top_p` 但两者不能同时存在，二选一。

- **`n`** （integer，选填，默认为 1）

  每个 `prompt` 生成的补全次数。

  注意：由于此参数会生成许多补全，因此它会快速消耗token配额。小心使用，并确保对 `max_tokens` 和 `stop` 进行合理的设置。


## 生成英文文本

In [ ]:
import os
from openai import OpenAI

client = OpenAI()

data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

In [ ]:
print(data)

In [ ]:
text = data.choices[0].text

In [ ]:
print(text)

## 生成中文文本

调整 `max_tokens` 

In [ ]:
data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="讲10个给程序员听得笑话",
  max_tokens=1000,
  temperature=0.5
)

In [ ]:
text = data.choices[0].text
print(text)

## 生成 Python 代码，并执行和验证

以面试中考察的典型的试题 `快速排序` 为例

In [ ]:
data = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="生成可执行的快速排序 Python 代码",
  max_tokens=1000,
  temperature=0
)


In [ ]:
text = data.choices[0].text
print(text)

#### Prompt：Jupyter Notebook 中执行生成的代码

Prompt：

```
我现在用 Completion API 生成了 Python 代码，并以字符串形式存放在 text 中，如下所示：

text = data.choices[0].text
print(text)

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[0]
    left = [x for x in arr[1:] if x <= pivot]
    right = [x for x in arr[1:] if x > pivot]
    return quick_sort(left) + [pivot] + quick_sort(right)

如何在 Jupyter notebook 中执行text中存放的这段代码
```


In [ ]:
# `exec` 函数会执行传入的字符串作为 Python 代码。
# 在这个例子中，我们使用 `exec` 来定义了一个 `quick_sort` 函数，然后你就可以调用这个函数了。
# 请注意，`exec` 可以执行任何 Python 代码，因此在使用它的时候一定要小心，特别是当你执行的代码来自不可信的来源时。
exec(text)

In [ ]:
# 现在你可以调用这个函数了
print(quick_sort([12,3,6,8,10,1,2,1]))

# 聊天机器人初探（Chat Completions API）

使用 Chat Completions API 实现对话任务

聊天补全(Chat Completions API)以消息列表作为输入，并返回模型生成的消息作为输出。尽管聊天格式旨在使多轮对话变得简单，但它同样适用于没有任何对话的单轮任务。

主要请求参数说明：


- **`model` （string，必填）**

  要使用的模型ID。有关哪些模型适用于Chat API的详细信息

- **`messages` （array，必填）**

  迄今为止描述对话的消息列表
    - **`role` （string，必填）**

  发送此消息的角色。`system` 、`user` 或 `assistant` 之一（一般用 user 发送用户问题，system 发送给模型提示信息）

    - **`content` （string，必填）**
    
      消息的内容
    
    - **`name` （string，选填）**
    
      此消息的发送者姓名。可以包含 a-z、A-Z、0-9 和下划线，最大长度为 64 个字符

- **`stream` （boolean，选填，是否按流的方式发送内容）**

  当它设置为 true 时，API 会以 SSE（ Server Side Event ）方式返回内容。SSE 本质上是一个长链接，会持续不断地输出内容直到完成响应。如果不是做实时聊天，默认false即可。

- **`max_tokens` （integer，选填）**

  在聊天补全中生成的最大 **tokens** 数。

  输入token和生成的token的总长度受模型上下文长度的限制。

- **`temperature` （number，选填，默认是 1）**

  采样温度，在 0和 2 之间。

  较高的值，如0.8会使输出更随机，而较低的值，如0.2会使其更加集中和确定性。

  通常建议修改这个（`temperature` ）或者 `top_p` ，但两者不能同时存在，二选一。


## 开启聊天模式

使用 `messages` 记录迄今为止对话的消息列表

In [ ]:
from openai import OpenAI
client = OpenAI()

messages=[
    {
        "role": "user", 
        "content": "Hello!"
    }
]


data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages = messages
)


In [ ]:
print(data)

In [ ]:
# 从返回的数据中获取生成的消息
new_message = data.choices[0].message
# 打印 new_message
print(new_message)

In [ ]:
# 将消息追加到 messages 列表中
messages.append(new_message)
print(messages)

In [ ]:
type(new_message)

In [ ]:
new_message.role

In [ ]:
new_message.content

In [ ]:
messages.pop()

In [ ]:
print(messages)

#### Prompt: OpenAIObject -> Dict

```
打印 messages 列表后发现数据类型不对，messages 输出如下：

print(messages)

[{'role': 'user', 'content': 'Hello!'}, <OpenAIObject at 0x7f27582c13f0> JSON: {
  "content": "Hello! How can I assist you today?",
  "role": "assistant"
}]

将OpenAIObject 转换为一个如下数据类型格式：

    {
        "role": "user", 
        "content": "Hello!"
    }
```

In [ ]:
new_message = data.choices[0].message
new_message_dict = {"role": new_message.role, "content": new_message.content}
type(new_message_dict)

In [ ]:
print(new_message_dict)

In [ ]:
# 将消息追加到 messages 列表中
messages.append(new_message_dict)

In [ ]:
print(messages)

#### 新一轮对话

In [ ]:
new_chat = {
    "role": "user",
    "content": "1.讲一个程序员才听得懂的冷笑话；2.今天是几号？3.明天星期几？"
}

In [ ]:
messages.append(new_chat)

In [ ]:
from pprint import pprint

pprint(messages)

In [ ]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [ ]:
new_message = data.choices[0].message
# 打印 new_messages 
print(new_message)

In [ ]:
# 打印 new_messages 内容
print(new_message.content)

## 使用多种身份聊天对话

目前`role`参数支持3类身份： `system`, `user` `assistant`:


![](images/chat_completion_api.png)



In [ ]:
# 构造聊天记录
messages=[
    {"role": "system", "content": "你是一个乐于助人的体育界专家。"},
    {"role": "user", "content": "2008年奥运会是在哪里举行的？"},
]

In [ ]:
import openai

data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)


In [ ]:
message = data.choices[0].message.content
print(message)

In [ ]:
# 添加 GPT 返回结果到聊天记录
messages.append({"role": "assistant", "content": message})

In [ ]:
messages

In [ ]:
# 第二轮对话
messages.append({"role": "user", "content": "1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？"})

In [ ]:
messages

In [ ]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

In [ ]:
message = data.choices[0].message.content
print(message)

In [ ]:
data = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[{'role': 'user', 'content': '1.金牌最多的是哪个国家？2.奖牌最多的是哪个国家？'}]
)

In [ ]:
data.choices[0].message.content